## READ FILE


In [12]:
import numpy as np
import pandas as pd

class ProductionData:
    def __init__(self, demand_forecast, setup_cost, production_cost, holding_cost, num_periods):
        self.demand_forecast = np.array(demand_forecast)
        self.setup_cost = np.array(setup_cost)
        self.production_cost = np.array(production_cost)
        self.holding_cost = np.array(holding_cost)
        self.num_periods = num_periods
    
    def __repr__(self):
        return (f"ProductionData(\n"
                f"  Demand Forecast: {self.demand_forecast}\n"
                f"  Setup Cost: {self.setup_cost}\n"
                f"  Production Cost: {self.production_cost}\n"
                f"  Holding Cost: {self.holding_cost}\n"
                f"  Number of Periods: {self.num_periods}\n")

def read_production_data(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Insert dummy period 0
    demand_forecast = np.insert(df["Demand Forecast"].to_numpy(), 0, 0)
    setup_cost = np.insert(df["Setup Cost"].to_numpy(), 0, 0)
    production_cost = np.insert(df["Production cost"].to_numpy(), 0, 0)
    holding_cost = np.insert(df["Holding cost"].to_numpy(), 0, 0)
    
    num_periods = df.shape[0]
    
    return ProductionData(demand_forecast, setup_cost, production_cost, holding_cost, num_periods)


In [20]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# Define file path and sheet names
file_path = "DATA_corrected.xlsx"
sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]

# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S"])

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    data=read_production_data(file_path, sheet_name)
    T=data.num_periods
    
    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")

    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  

    # Objective function
    model.setObjective(
        quicksum(data.setup_cost[t] * y[t] for t in range(1, T+1)) +
        quicksum(data.production_cost[t] * x[t] for t in range(1, T+1)) +
        quicksum(data.holding_cost[t] * S[t] for t in range(1, T+1)),
        GRB.MINIMIZE
    )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")

    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] == data.demand_forecast[t] + S[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= quicksum(data.demand_forecast[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")

    # Solve model
    model.optimize()

    # Store results in DataFrame
    if model.status == GRB.OPTIMAL:
        for t in range(1, T+1):
            results_df = pd.concat([results_df, pd.DataFrame({
                "Sheet": [sheet_name],
                "Period": [t],
                "y": [y[t].X],
                "x": [x[t].X],
                "S": [S[t].X]
            })], ignore_index=True)


# Print all results at the end
print("\nAll results:")
print(results_df)

# Optionally save to a CSV file
results_df.to_csv("all_results.csv", index=False)
print("\nResults saved to 'all_results.csv'.")




Processing sheet: 6-periods (1)
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 21 columns and 32 nonzeros
Model fingerprint: 0x0695a2d9
Variable types: 14 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 7 rows, 11 columns, 17 nonzeros
Variable types: 7 continuous, 4 integer (4 binary)
Found heuristic solution: objective 54875.000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

C:\Users\ricca\AppData\Local\Temp\ipykernel_9484\1790507743.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({



CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 39 columns and 62 nonzeros
Model fingerprint: 0x08bd9955
Variable types: 26 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [2e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+03]
Found heuristic solution: objective 45218.000000
Presolve removed 6 rows and 8 columns
Presolve time: 0.00s
Presolved: 20 rows, 31 columns, 50 nonzeros
Variable types: 20 continuous, 11 integer (11 binary)

Root relaxation: objective 4.151611e+04, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 41516.1059    0    8 45218.0000 41516.1059  8.19%     -    0s
 

## Wagner within


In [82]:
import numpy as np

def wagner_whitin_nobacklog(demand, setup_costs, holding_costs, unit_costs, num_periods):
    T = num_periods  # Number of periods
    C = np.full(T+1, float('inf'))  # Cost array, initialized to infinity
    C[0] = 0  # Base case: No cost at period 0
    order_periods = [-1] * (T+1)  # To track order placements
    
    # Compute minimum cost for each period t
    for t in range(1, T+1):
        total_holding_cost = 0



    for t in range(1, T):
        for j in range(t, 0, -1):  # Consider ordering at period j
            # Calculate total demand from j to t
            total_demand = sum(demand_forecast[j:t])

            # Holding cost should consider storage time: (holding cost per period * quantity stored * number of periods stored)
            total_holding_cost = sum(
                holding_cost[k] * sum(demand_forecast[k:t])
                for k in range(j, t)
            )

            # Production cost is just the cost of producing the total demand
            production_cost = unit_costs[j] * total_demand
            
            # Compute total cost
            cost = C[j - 1] + setup_cost[j] + total_holding_cost + production_cost

            if cost < C[t]:  # Store minimum cost decision
                C[t] = cost
                order_periods[t] = j

    # Backtrack to find optimal order schedule
    t = T
    orders = []
    while t > 0:
        j = order_periods[t]  # Retrieve order period
        orders.append((j, sum(demand[j - 1:t])))  # Store (order time, quantity ordered)
        t = j - 1  # Move to the previous order period
    
    return list(reversed(orders)), C[T]  # Reverse for correct order sequence

for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    data=read_production_data(file_path, sheet_name)
    #print(data.demand_forecast[0])
    #print(data.holding_cost[0])

    optimal_orders, min_cost = wagner_whitin_nobacklog(data.demand_forecast, data.setup_cost, data.holding_cost, data.production_cost, data.num_periods)
    print("Optimal order schedule:", optimal_orders)
    print("Minimum total cost:", min_cost)

    
    
    




Processing sheet: 6-periods (1)
Optimal order schedule: [(-1, 456)]
Minimum total cost: inf

Processing sheet: 6-periods (2)
Optimal order schedule: [(-1, 818)]
Minimum total cost: inf

Processing sheet: 12-periods (1)
Optimal order schedule: [(-1, 506)]
Minimum total cost: inf

Processing sheet: 12-periods (2)
Optimal order schedule: [(-1, 0)]
Minimum total cost: inf

Processing sheet: 24-periods (1)
Optimal order schedule: [(-1, 783)]
Minimum total cost: inf

Processing sheet: 24-periods (2)
Optimal order schedule: [(-1, 645)]
Minimum total cost: inf

Processing sheet: 52-periods (1)
Optimal order schedule: [(-1, 157)]
Minimum total cost: inf

Processing sheet: 52-periods (2)
Optimal order schedule: [(-1, 157)]
Minimum total cost: inf

Processing sheet: 104-periods (1)
Optimal order schedule: [(-1, 63)]
Minimum total cost: inf

Processing sheet: 104-periods (2)
Optimal order schedule: [(-1, 653)]
Minimum total cost: inf
